In [6]:
from time import time
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import astropy.units as u
import astropy.constants as c
import astropy.io.fits as pf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import gpr
import mcmc
import forAustin as fa

In [7]:
sample = np.array([-0.5, 0, 0, 0.5])
# sample = None
GP = gpr.GPR(
    '/media/data/austinfortino/austinFull.fits',
    500,
    sample=sample,
    verbose=False,
    eps=0,
    test_size=0.2,
    random_state=0,
    tensor=True)

In [8]:
def LML(theta, GP=GP):
    GP.fit(theta)
    LML = GP.get_LML()
    return LML

In [9]:
var_s = 500
sigma_x = 0.01
sigma_y = 0.01
phi = (0 * u.deg).to(u.rad).value
theta0 = np.array([var_s, sigma_x, sigma_y, phi])
step_size = np.array([5, 5e-4, 5e-4, 1e-7])

In [10]:
samples, _ = tfp.mcmc.sample_chain(
    num_results=1,
    current_state=theta0,
    kernel=tfp.mcmc.RandomWalkMetropolis(
        LML,
        seed=42),
    trace_fn=None,
    num_burnin_steps=0,
    parallel_iterations=1)

sample_mean = tf.math.reduce_mean(samples, axis=0)
sample_std = tf.sqrt(
    tf.math.reduce_mean(tf.squared_difference(samples, sample_mean),
                   axis=0))
with tf.Session() as sess:
  [sample_mean_, sample_std_] = sess.run([sample_mean, sample_std])

print('Estimated mean: {}'.format(sample_mean_))
print('Estimated standard deviation: {}'.format(sample_std_))

TypeError: Tensor objects are only iterable when eager execution is enabled. To iterate over this tensor use tf.map_fn.